# Combining Theories

To generalize all of these results, we need to combine the calculations and drive them from model data from our actual design. There is a significant problem with doing that now. Mcleans estimation scheme makes not reference to flying surface alignment, which significantly affects the lift and drag forces produced by those surfaces. And Erbach only provided power required for level flight as a function of the surface alignment, so we have no idea what settings to use when calculating flight time. We will have to deal with that issue before we reach our conclusion. For now, let's focus on generalizing Erbach's calculations so we can drive it with arbitrary model data.

## Calculating Geometry from Plans

We need to be able ot calculate the moment arm lengths for areal model flying at some angle of attach. Basically, i want to create a Python function that can do that using input model data.

Here are our four moment arms:


\begin{equation}
y_{mw} = h_w \cos{\alpha}- x_{cg} \sin{\alpha}
\end{equation}

\begin{equation}
x_{mw} = h_w \sin{\alpha} + x_{cg} \sin{\alpha}
\end{equation}

\begin{equation}
y_{ms} =  (x_{ac} - x_{cg}) \sin{\alpha}
\end{equation}

\begin{equation}
x_{ms} =  (x_{ac} - x_{cg}) \cos{\alpha}
\end{equation}


The wing height is simple. From the **Wart** plan

In [3]:
import pint
u = pint.UnitRegistry()

In [4]:
h_w = 7.0/8.0 * u.inch

One obvious simplification is to use the velocity and power calculations from Erbach's code in place of th calculations made by McLean. To get those values, we need to generalize Erbach's program so we can feed it data from any model design. Fortunately, this is not too hard, given that w can find suitable aerodynamic coefficient data for the chosen airfoil.

For circular arc airfoils, I was lucky enough to find some experimental data from Japan!

The location of the center of gravity is not specified in the plan, so we will guess for this experiment. Since we are assuming that the aerodynamic center is located at 25% cord, that gives us a value for $x_{cg}$ of 0.5 times the chord of the wing.  

In [45]:
wart_best_time = 618 * u.seconds
wart_airframe_weight = 1.2 * u.grams
wart_motor_weight = 0.86 * u.grams
wart_total_weight = wart_airframe_weight + wart_motor_weight
wart_wing_span = (8.0 + 2*3.65)*u.inch
wart_stab_span = 10.8 * u.inch
wart_wing_area = 30.0 * u.inch**2
wart_stab_area = ((1.0 + 1.75)*5.4)*u.inch**2
wart_stab_mean_chord = wart_stab_area / wart_stab_span
wart_wing_mean_chord = wart_wing_area / wart_wing_span
wart_propeller_radius = 3.0 * u.inch
motor_turns = 3660
kiby_dome_elevation = 2160 * u.feet
kiby_dome_height = 147 * u.feet

In [46]:
x_cg = 0.5 * wart_wing_mean_chord
x_cg

0.9803921568627451 <Unit('inch')>

The distance between aerodynamic centers, $x_{ac}$ is calculated next. We need a few more dimensions from the plan:

In [47]:
wart_motor_stick_length = 6.0 * u.inch
wart_wing_offset = 1.25 * u.inch
wart_tailboom_length = 8.5 * u.inch

In [48]:
x_ac = (wart_motor_stick_length + wart_tailboom_length) \
        - 0.25 *(wart_wing_mean_chord + wart_stab_mean_chord)
x_ac

13.666053921568627 <Unit('inch')>

We have the moment arm values now, but the moment calculations depend on the aerodynamic properties of the wing and stab, which in turn depend on the flight angle of attack. From the wart plan, both wing and stab were initially set at zero degrees. McLean assumed that the wing was operating at a lift coefficient of 1.0, and for the circular arc I am using, thst happens at about six degrees angle of attach so we can use the same coefficient values for  so lets pick coefficient f=data for that angle of attack:

In [49]:
C_lw = 0.885      # approximate for now
C_dw = 0.0789
C_ls = 0.500    # WAG _ need better data for 2% circular arc
C_ds = 0.0500

In [50]:
S_w = wart_wing_area
S_s = wart_stab_area

In [51]:
M_w = wart_total_weight
rho = 0.00231683 * u.slugs/u.ft**3 # standard atmosphere for KC

In [108]:
aerodynamic_center_wing = 0.25  * wart_wing_mean_chord
aerodynamic_center_stab = 0.25 * wart_stab_mean_chord
pitching_moment_wing = -0.10
pitching_moment_stab = 0.05
l = ((6 - 1.25)*u.inch - aerodynamic_center_wing) + ((8.5 - 1.75)*u.inch + aerodynamic_center_stab)
x = (1.25 + 2.0) * u.inch
clr1 = - x/aerodynamic_center_wing 
clr2 = pitching_moment_wing 
clr3 = pitching_moment_stab * \
        (wart_stab_area * aerodynamic_center_stab) \
        / (wart_wing_area * aerodynamic_center_wing)
clr4 =  1.0  + x/l
clr5 = (wart_stab_area * l)/(wart_wing_area*aerodynamic_center_wing)
C_lr = (clr1 + clr2 + clr3)/(clr4 * clr5)
print(C_lr)

-0.45519731786882367 dimensionless


In [109]:
C_l = (C_lw + (wart_stab_area * C_lr)/ wart_wing_area) / \
      (C_lw + (wart_stab_area / wart_wing_area))
print(C_l)

0.4780270490253132 dimensionless


In [113]:
V = (2.0 *(M_w/(rho/u.gravity * C_l*(S_w+S_s))))**0.5
V.to('miles/hour')

3.498734749113699 <Unit('mile / hour')>

In [52]:
u_l = (2*(M_w/(rho/u.gravity*(C_ls*S_s + C_lw*S_w)))) **0.5
u_l.to_base_units()

1.2424677639177757 <Unit('meter / second')>

In [53]:
u_l.to('ft/sec')

4.076337808129185 <Unit('foot / second')>

In [54]:
u_l.to('miles/hour')

2.779321232815353 <Unit('mile / hour')>

That is more in line with Gary's estimation of the flight speed!

Now, on to the power requirements.

In [55]:
q = rho * u_l**2/2.0 / u.gravity
q.to_base_units()

0.09398105624761657 <Unit('kilogram / meter ** 2')>

In [56]:
D_t = q * (S_w * C_dw + S_s * C_ds)
power = D_t * u_l
power.to('in oz/sec')

0.3253149120485764 <Unit('inch * ounce / second')>

We will ignore the profile and induced drag calculations for the moment.

In [57]:
k_m = 30000 * u.inches
E = k_m * wart_motor_weight
energy_to_power_ratio = E/power
energy_to_power_ratio.to_base_units()

2797.4992371800113 <Unit('second')>

In [60]:
D_t

0.1885377483443709 <Unit('gram')>

In [68]:
q = rho /u.gravity * u_l**2/2.0
q

0.0606328182487123 <Unit('gram / inch ** 2')>

In [85]:
s_p = math.pi * wart_propeller_radius ** 2
s_p

28.274333882308138 <Unit('inch ** 2')>

In [87]:
tau = D_t / (q*s_p)
tau

0.10997606567649969 <Unit('dimensionless')>

In [88]:
omega = motor_turns/wart_best_time
omega

5.922330097087379 <Unit('1 / second')>

In [89]:
J = u_l / (omega * wart_propeller_radius)
J.to_base_units()

2.7531986507364334 <Unit('dimensionless')>

In [90]:
tau

0.10997606567649969 <Unit('dimensionless')>

In [91]:
eta_i = (2 - J**2*tau/math.pi**2)/(math.sqrt(tau + 1.0) + 1.0)
eta_i.to_base_units()

0.9327904783577515 <Unit('dimensionless')>

In [100]:
epsilon = 0.1 # assumed from McLean
eta_p = eta_i * (1.0-(4.0*J*epsilon)/(3.0*math.pi*eta_i))/ \
                (1.0+(2.0*eta_i*math.pi*epsilon)/(3.0*J))
eta_p

0.761879200650401 <Unit('dimensionless')>

In [101]:
H = kiby_dome_height/(483 * u.foot * wart_motor_weight/wart_total_weight)
H

0.7290192113245703 <Unit('dimensionless')>

In [102]:
F = 0.75
eta_av = F * eta_p
eta_av

0.5714094004878008 <Unit('dimensionless')>

In [103]:
calculated_time = eta_av * E / power
calculated_time.to_base_units()

1598.5173619821098 <Unit('second')>

This is clearly not a good number, off by almost a factor of three. Where did we go wrong?

The value of $\eta_{av}$ we should have found it this:


In [104]:
eta_cav = wart_best_time * power / E
eta_cav.to_base_units()

0.2209115883881235 <Unit('dimensionless')>

Is the fudge factor wrong? Probably, but there is no example data to compare to so we guessed a value. Let's keep working backward. 

In [107]:
F_g = eta_cav/eta_p
F_g.to_base_units()

0.2899561875419826 <Unit('dimensionless')>

In [ ]:
This value is not going to fall on McLean's chart. Perhaps the power calculaiton is off. 